In [1]:
import tensorflow as tf
# import local version of pysyft
import sys
sys.path.append("../..")
import syft as sy

In [2]:
import random
import numpy as np

In [3]:
hook = sy.TorchHook()

In [4]:
# test dataset
XOR_X = np.array([[0,0],[0,1],[1,0],[1,1]])
XOR_Y = np.array([[0],[1],[1],[0]])

# Before PySyft

In [ ]:
# architecture
def multilayer_perceptron(x, weights, biases, keep_prob):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    out_layer = tf.sigmoid(tf.matmul(layer_1, weights['out']) + biases['out'])
    return out_layer

In [ ]:
# build net
n_hidden_1 = 2
n_input = XOR_X.shape[1]
n_classes = XOR_Y.shape[1]

weights = {
    'h1': tf.Variable(tf.random_uniform(shape=[n_input, n_hidden_1],
                                       minval=-1,
                                       maxval=1),
                      name="Theta1"),
    'out': tf.Variable(tf.random_uniform(shape=[n_hidden_1, n_classes],
                                        minval=-1,
                                        maxval=1),
                       name="Theta2")
}


biases = {
    'b1': tf.Variable(tf.zeros([n_hidden_1]),
                      name="Bias1"),
    'out': tf.Variable(tf.zeros([n_classes]),
                       name="Bias2")
}

# training params

n_epochs = 10000
display_step = 1000
batch_size = 1

# set up placeholders
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

# set up net symbolic tensors
predictions = multilayer_perceptron(x, weights, biases, keep_prob)
cost = tf.reduce_mean(( (y * tf.log(predictions)) + ((1 - y) * tf.log(1.0 - predictions)) ) * -1)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-2).minimize(cost)

In [ ]:
# train

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(n_epochs+1):
        avg_cost = 0.0

        _, c = sess.run([optimizer, cost],
                        feed_dict={
                            x: XOR_X, 
                            y: XOR_Y,
                        })
        avg_cost += c / n_epochs
        if epoch % display_step == 0:
            print("Epoch: {:04d}\nCost: {:.9f}".format(epoch, avg_cost))
    
    correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("\nAccuracy: {:.2f}".format(accuracy.eval({x: XOR_X, y: XOR_Y})))

# With PySyft

In [5]:
me = hook.local_worker
me.id = "me" # necessary for building Syft Tensor, in get_tf_override_function this must be a string

In [6]:
me.is_client_worker = False

In [7]:
bob = sy.VirtualWorker(id='bob')
alice = sy.VirtualWorker(id='alice')

bob.add_worker(me)
bob.add_worker(alice)

me.add_worker(bob)
me.add_worker(alice)

alice.add_worker(me)
alice.add_worker(bob)

In [8]:
def get_tf_override_function(attr):
    def method(*args, **kwargs):

        worker = None
        if('worker' in kwargs):
            worker = kwargs['worker']
            del kwargs['worker']        
        
        kwargs['name'] = worker.id + "_" + kwargs['name'] + "_" + str(random.randint(0,10e10))
        
        cmd, locations, owners = sy.core.frameworks.torch.utils.compile_command(
                                                        attr=str(attr),
                                                        args=args,
                                                        kwargs=kwargs,
                                                        has_self=False,
                                                        self=None
                                                    )
        if(len(locations) > 0):
            worker = locations[0]
        elif worker is not None:
            ""
        else:
            worker = sy.local_worker

        result = sy.local_worker.send_command(recipient=worker,
                                              message=cmd,
                                              framework="tf")
        return result
        
    return method

In [9]:
tf.Tensor.native__init__ = tf.Tensor.__init__
def new___init__(self, *args, **kwargs):
    print("initializing")

    if 'owner' in kwargs and kwargs['owner'] is not None:
        owner = kwargs['owner']
    else:
        owner = hook.local_worker
    
    self.native__init__(*args, *kwargs)
    
    owner.register_object(self)    
    
tf.Tensor.__init__ = new___init__

@property
def id(self):
    return self.name

tf.Tensor.id = id

@property
def owner(self):
    return sy.local_worker.get_worker(self.name.split("_")[0])

tf.Tensor.owner = owner

def create_pointer(self):
    return self

tf.Tensor.create_pointer = create_pointer

In [10]:
result = get_tf_override_function('placeholder')(tf.float32, shape=[4,2], name="x-input", worker=bob)

processing tf call
{'command': 'placeholder', 'has_self': False, 'args': ('tf.float32',), 'kwargs': {'shape': [4, 2], 'name': 'bob_x-input_34038910726'}}
process tf.function
initializing


In [16]:
result_alice = get_tf_override_function('placeholder')(tf.float32, shape=[4,2], name="x-input", worker=alice)

processing tf call
{'command': 'placeholder', 'has_self': False, 'args': ('tf.float32',), 'kwargs': {'shape': [4, 2], 'name': 'alice_x-input_4383544955'}}
process tf.function
initializing


In [17]:
bob._objects

{'bob_x-input_34038910726:0': <tf.Tensor 'bob_x-input_34038910726:0' shape=(4, 2) dtype=float32>}

In [18]:
alice._objects

{'alice_x-input-alice_3457222468:0': <tf.Tensor 'alice_x-input-alice_3457222468:0' shape=(4, 2) dtype=float32>,
 'alice_x-input_4383544955:0': <tf.Tensor 'alice_x-input_4383544955:0' shape=(4, 2) dtype=float32>}

In [19]:
me._objects

{}

In [20]:
result_alice

{'thing': 'TensorPointer'}

In [21]:
class PointerVariable(tf.Variable):
    #def __add__(self, other):
    #    return self.__sub__(other)
    def __new__(self, *args, **kwargs):
        super().__init__(self, *args, **kwargs)
        print ("initializing")

In [22]:
pv = PointerVariable(tf.zeros([2]), name="pointer")

initializing


--- Logging error ---
Traceback (most recent call last):
  File "/home/sam/miniconda3/envs/pysyft/lib/python3.6/logging/__init__.py", line 994, in emit
    msg = self.format(record)
  File "/home/sam/miniconda3/envs/pysyft/lib/python3.6/logging/__init__.py", line 840, in format
    return fmt.format(record)
  File "/home/sam/miniconda3/envs/pysyft/lib/python3.6/logging/__init__.py", line 577, in format
    record.message = record.getMessage()
  File "/home/sam/miniconda3/envs/pysyft/lib/python3.6/logging/__init__.py", line 338, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/home/sam/miniconda3/envs/pysyft/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/sam/miniconda3/envs/pysyft/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/sam/miniconda3/envs/pysyft/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <mod

AttributeError: 'str' object has no attribute 'id'

In [ ]:
x_ = tf.placeholder(tf.float32, shape=[4,2], name="x-input")
y_ = tf.placeholder(tf.float32, shape=[4,1], name="y-input")

In [ ]:
x_

In [ ]:
Theta1 = tf.Variable(tf.random_uniform([2,2], -1, 1), name="Theta1")
Theta2 = tf.Variable(tf.random_uniform([2,1], -1, 1), name="Theta2")

In [ ]:
Bias1 = tf.Variable(tf.zeros([2]), name="Bias1")
Bias2 = tf.Variable(tf.zeros([1]), name="Bias2")

In [ ]:
A2 = tf.sigmoid(tf.matmul(x_, Theta1) + Bias1)
Hypothesis = tf.sigmoid(tf.matmul(A2, Theta2) + Bias2)

In [ ]:
cost = tf.reduce_mean(( (y_ * tf.log(Hypothesis)) + 
        ((1 - y_) * tf.log(1.0 - Hypothesis)) ) * -1)

In [ ]:
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cost)

In [ ]:
XOR_X = [[0,0],[0,1],[1,0],[1,1]]
XOR_Y = [[0],[1],[1],[0]]

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [ ]:
for i in range(100000):
    sess.run(train_step, feed_dict={x_: XOR_X, y_: XOR_Y})
    
    if i % 1000 == 0:
        print('Epoch ', i)
#         print('Hypothesis ', sess.run(Hypothesis, feed_dict={x_: XOR_X, y_: XOR_Y}))
#         print('Theta1 ', sess.run(Theta1))
#         print('Bias1 ', sess.run(Bias1))
#         print('Theta2 ', sess.run(Theta2))
#         print('Bias2 ', sess.run(Bias2))
        print('cost ', sess.run(cost, feed_dict={x_: XOR_X, y_: XOR_Y}))